# Azure Open AI o1 multi-modal Test
The following sample shows the most basic way to use the o1(GA) with Vision model with code.

> ✨ ***Note*** <br>
> Please check the supported models and API version before you get started - https://azure.microsoft.com/en-us/blog/announcing-the-o1-model-in-azure-openai-service-multimodal-reasoning-with-astounding-analysis/?msockid=388843f556c46b710353575a57e66a9a

## Prerequisites
Configure a Python virtual environment for 3.10 or later: 
 1. open the Command Palette (Ctrl+Shift+P).
 1. Search for Python: Create Environment.
 1. select Venv / Conda and choose where to create the new environment.
 1. Select the Python interpreter version. Create with version 3.10 or later.

For a dependency installation, run the code below to install the packages required to run it. 

```bash
pip install -r requirements.txt
```

## Set up your environment
Git clone the repository to your local machine. 

```bash
git clone https://github.com/hyogrin/Azure_OpenAI_samples.git
```

Create an .env file based on the .env-sample file. Copy the new .env file to the folder containing your notebook and update the variables.

## 🔨 Current Support and Limitations (as of 2025-01-06) 
- In order to use the image processing, please understand that you must call the o1 using API at this moment.
- GA model version is 2024-12-17. Please check the deployed model version before calling
- Model o1 is enabled only for api versions **2024-12-01-preview** and later. Please set the api version currently. 
- Check the region o1 supported - https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference#o1-modelsopen ai rest
- Check the API version o1 supported - https://learn.microsoft.com/en-us/azure/ai-services/openai/reference-preview

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys
module_path = "../util"
sys.path.append(os.path.abspath(module_path))

from common import check_kernel
check_kernel()

Kernel: python31015jvsc74a57bd07fae5fd5097eba353202a8ff671bfdb53a1767393571418b923386fbe14d03af


## 🧪 o1 multimodal with image url 

In [2]:
import os
import json
import openai
import base64
from openai import AzureOpenAI
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, EnvironmentCredential
from azure.keyvault.secrets import SecretClient
from io import BytesIO
import gradio as gr
load_dotenv(override=True)

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
#azure_openai_key = os.getenv("AZURE_OPENAI_KEY", "") if len(os.getenv("AZURE_OPENAI_KEY", "")) > 0 else None
azure_openai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-ada-002")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "") if len(os.getenv("AZURE_OPENAI_API_VERSION", "")) > 0 else None
kv_secret_name = os.getenv("KEY_VAULT_SECRET_NAME")
tenant_id = os.getenv("AZURE_TENANT_ID")


# Use DefaultAzureCredential to authenticate with Azure Key Vault
# credential = DefaultAzureCredential()
credential = InteractiveBrowserCredential(tenant_id=tenant_id)

key_vault_url = os.getenv("AZURE_KEY_VAULT_URL")
key_vault_secret_name = os.getenv("AZURE_KEY_VAULT_SECRET_NAME")

kv_client = SecretClient(vault_url=key_vault_url, credential=credential)

# Retrieve the OpenAI API key from Key Vault
azure_openai_key = kv_client.get_secret(key_vault_secret_name).value

# Initialize the AzureOpenAI client with the retrieved key
try:
    client = AzureOpenAI(
        azure_endpoint=azure_openai_endpoint,
        api_key=azure_openai_key,
        api_version=aoai_api_version
    )
except (ValueError, TypeError) as e:
    print(e)

/afh/code/Azure_OpenAI_samples/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
response = client.chat.completions.create(
    model=azure_openai_deployment_name,
    messages=[
        { "role": "user", "content": [  
            { 
                "type": "text", 
                "text": "Describe this picture:" 
            },
            { 
                "type": "image_url",
                "image_url": {
                    "url": "https://devblogs.microsoft.com/semantic-kernel/wp-content/uploads/sites/78/2023/09/semantic-kernel-in-prompt-flow-1.png"
                }
            }
        ] } 
    ],
    
)
print(response)
print(response.choices[0].message.content)
print("Usage Information:")
print(f"Cached Tokens: {response.usage.prompt_tokens_details.cached_tokens}")
print(f"Completion Tokens: {response.usage.completion_tokens}")
print(f"Prompt Tokens: {response.usage.prompt_tokens}")
print(f"Total Tokens: {response.usage.total_tokens}")

ChatCompletion(id='chatcmpl-Ams6s31VOVX2AGrzFyFFVLkIYcijB', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='It’s a simple vertical flow diagram showing data (“Benchmark data”) entering a “Prompt flow” box. Inside that box, there are three steps in sequence: “Input,” the pink “Semantic Kernel,” and then “Output.” From there, the flow continues downward to a final box labeled “Results and metrics.” Essentially, it illustrates a pipeline where benchmark data feeds into prompt processing by the Semantic Kernel to produce output and, ultimately, measurable results.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'seve

## 🧪 o1 multimodal with local image (base64 encoding)

In [23]:
import base64
from mimetypes import guess_type

# Function to encode a local image into data URL 
def local_image_to_data_url(image_path):
    # Guess the MIME type of the image based on the file extension
    mime_type, _ = guess_type(image_path)
    if mime_type is None:
        mime_type = 'application/octet-stream'  # Default MIME type if none is found

    # Read and encode the image file
    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Construct the data URL
    return f"data:{mime_type};base64,{base64_encoded_data}"

# Example usage
image_path = './images/semantic-kernel-in-prompt-flow-1.png'
data_url = local_image_to_data_url(image_path)
#print("Data URL:", data_url)

In [10]:
response = client.chat.completions.create(
    model=azure_openai_deployment_name,
    messages=[
        { "role": "user", "content": [  
            { 
                "type": "text", 
                "text": "Describe this picture:" 
            },
            { 
                "type": "image_url",
                "image_url": {
                    "url": data_url 
                }
            }
        ] } 
    ],
    
)
print(response)
print(response.choices[0].message.content)
print("Usage Information:")
print(f"Cached Tokens: {response.usage.prompt_tokens_details.cached_tokens}")
print(f"Completion Tokens: {response.usage.completion_tokens}")
print(f"Prompt Tokens: {response.usage.prompt_tokens}")
print(f"Total Tokens: {response.usage.total_tokens}")

ChatCompletion(id='chatcmpl-Amd0V1QeEmGbu1u9PbiW23TuO0kr9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='It is a simple, vertical flowchart showing how data moves through a “prompt flow” to produce results. At the top, a rounded rectangle labeled “Benchmark data” feeds into a larger box titled “Prompt flow.” Inside that box, the order is:  \n• A smaller rectangle labeled “Input,”  \n• An arrow pointing down to a magenta oval labeled “Semantic Kernel,”  \n• Another arrow pointing down to a smaller rectangle labeled “Output.”  \nFrom there, an arrow leads to a rounded rectangle at the bottom labeled “Results and metrics.” The overall diagram visually represents the flow from benchmark data in, through the semantic kernel, and out to final results and measurements.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'protected_mat

## 🧪 o1 structured output 
Structured outputs make a model follow a JSON Schema definition that you provide as part of your inference API call. This is in contrast to the older JSON mode feature, which guaranteed valid JSON would be generated, but was unable to ensure strict adherence to the supplied schema. Structured outputs is recommended for function calling, extracting structured data, and building complex multi-step workflows.

### Supported models (as of 2025-01-06) 
- o1 version: 2024-12-17
- gpt-4o-mini version: 2024-07-18
- gpt-4o version: 2024-08-06

In [6]:
from pydantic import BaseModel


class GetInformations(BaseModel):
    weather_location: str
    country_location: str
    country_language: str


tools = [openai.pydantic_function_tool(GetInformations)]

response = client.chat.completions.create(
    model=azure_openai_deployment_name,
    messages=[
        { "role": "user", "content": [  
            { 
                "type": "text", 
                "text": "Hello, what is the current weather for the capital of South Korea?"
            }
        ] } 
    ],
    tools=tools
)
print(response)
print(response.choices[0].message.tool_calls[0].function)
print(response.choices[0].message.tool_calls[0].function.arguments)
print("Usage Information:")
print(f"Cached Tokens: {response.usage.prompt_tokens_details.cached_tokens}")
print(f"Completion Tokens: {response.usage.completion_tokens}")
print(f"Prompt Tokens: {response.usage.prompt_tokens}")
print(f"Total Tokens: {response.usage.total_tokens}")

ChatCompletion(id='chatcmpl-Ams8cRyrea6lIGWyVb5zQwBjSuzFl', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_h4TRCXiPdclXyzbc2hPBoHY4', function=Function(arguments='{"weather_location":"Seoul","country_location":"South Korea","country_language":"Korean"}', name='GetInformations'), type='function')]), content_filter_results={})], created=1736211214, model='o1-2024-12-17', object='chat.completion', service_tier=None, system_fingerprint='fp_db33bdfc5e', usage=CompletionUsage(completion_tokens=112, prompt_tokens=76, total_tokens=188, completion_tokens_details=CompletionTokensDetails(audio_tokens=0, reasoning_tokens=64, accepted_prediction_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)), prompt_filter_results=[{'prompt_index': 0, 'content_filter_re

In [8]:
print(response.model_dump_json(indent=3))

{
   "id": "chatcmpl-Ams8cRyrea6lIGWyVb5zQwBjSuzFl",
   "choices": [
      {
         "finish_reason": "tool_calls",
         "index": 0,
         "logprobs": null,
         "message": {
            "content": null,
            "refusal": null,
            "role": "assistant",
            "audio": null,
            "function_call": null,
            "tool_calls": [
               {
                  "id": "call_h4TRCXiPdclXyzbc2hPBoHY4",
                  "function": {
                     "arguments": "{\"weather_location\":\"Seoul\",\"country_location\":\"South Korea\",\"country_language\":\"Korean\"}",
                     "name": "GetInformations"
                  },
                  "type": "function"
               }
            ]
         },
         "content_filter_results": {}
      }
   ],
   "created": 1736211214,
   "model": "o1-2024-12-17",
   "object": "chat.completion",
   "service_tier": null,
   "system_fingerprint": "fp_db33bdfc5e",
   "usage": {
      "completion_t

## 🚀 o1 playground sample  

In [14]:
import base64

def o1_webapp_gradio(pil_image, prompt):
    """
    o1 model integration for analyzing images.
    """
    # Save image to an in-memory buffer
    buffer = BytesIO()
    pil_image.save(buffer, format="JPEG")
    buffer.seek(0)

    # Encode the image to base64
    image_data = base64.b64encode(buffer.read()).decode('utf-8')
    image_url = f"data:image/jpeg;base64,{image_data}"

    # Construct the messages
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                { 
                    "type": "image_url",
                    "image_url": {
                        "url": image_url 
                    }
                }
            ],
        },
    ]

    # Call the Azure OpenAI API
    response = client.chat.completions.create(
        model=azure_openai_deployment_name,
        messages=messages
    )

    return response.choices[0].message.content

In [ ]:
image_url = "https://cdn-dynmedia-1.microsoft.com/is/image/microsoftcorp/UHFbanner-MSlogo?fmt=png-alpha&bfc=off&qlt=100,1"
logo = "<center> <img src= {} width=100px></center>".format(image_url)
title = "o1 model Playground"

inputs = [
    gr.Image(type="pil", label="Your image"),
    gr.Text(label="Your prompt", value="Describe this image:"),
]
outputs = [
    gr.Text(label=f"{azure_openai_deployment_name} results"),
]
theme = "gradio/monochrome"  # https://huggingface.co/spaces/gradio/theme-gallery

gp4o_webapp = gr.Interface(
    fn=o1_webapp_gradio,
    inputs=inputs,
    outputs=outputs,
    description=logo,
    title=title,
    theme=theme,
)

gp4o_webapp.launch(share=True)